# Projektarbeit Data Analytics

Sebastian Jana,
Sophie Jana

## Inhaltsverzeichnis
<a id ="inhaltsverzeichnis"></a>

[1. Aufgabe](#aufgabe1)

2. Aufgabe

3. Aufgabe

4. Aufgabe
5. Aufgabe

[6. Aufgabe](#aufgabe6)

[7. Quellenverzeichnis](#quellenverzeichnis)




### Aufgabe 1 (Datenvorbereitung)
<a id = "aufgabe1"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a) Einlesen der CSV Dateien, welche die Stromerzeugungsdaten und die Börsenstrompreise enthalten, als Dataframe df_hourly.

Nach dem ersten Zusammenführen haben wir festgestellt, dass die Datumsspalte je nach CSV-Datei unterschiedliche Namen für Sommerzeit und Winterzeit hat. Dies führt dazu, dass beim Zusammenführen zwei separate Spalten entstehen. Um dies zu vermeiden, prüfen wir bereits beim Einlesen der Dateien die Spaltennamen und führen sie zu einer einheitlichen Spalte „Datum“ im df_hourly zusammen


In [2]:
import pandas as pd
import numpy as np
import os
import glob

In [11]:
# Sources for reading csv files from one folder[1],[2]
path = './Daten/Strompreisdaten'
# List all files (.csv) in the given path
csv_files = glob.glob(os.path.join(path, "*csv"))

df_list = []
for i in range(len(csv_files)):
    try:
        df_temp = pd.read_csv(csv_files[i], sep = ",")
        for column in df_temp.columns:
            # Combine the date columns, by getting rid of the naming difference in csv source
            if 'Datum (MESZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MESZ)':'Datum'})
            elif 'Datum (MEZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MEZ)':'Datum'})
        df_list.append(df_temp)
    except Exception as err: 
        print("Fehler beim Einlesen des Files: ", err)
    
df_hourly = pd.concat(df_list)
df_hourly

,Datum,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
0,2021-08-16 00:00:00,16015.81,19041.64,68.19
1,2021-08-16 01:00:00,15629.83,19420.11,65.03
2,2021-08-16 02:00:00,15322.51,19738.75,62.08
3,2021-08-16 03:00:00,15026.53,20822.64,60.13
4,2021-08-16 04:00:00,14987.63,21578.78,60.82
...,...,...,...,...
163,2021-04-25 19:00:00,18362.77,18593.89,65.93
164,2021-04-25 20:00:00,19477.05,16565.88,67.88
165,2021-04-25 21:00:00,19214.45,17819.31,66.00
166,2021-04-25 22:00:00,18964.11,18205.01,64.98


b) Zur besseren anschließenden Analyse überführen wir die Datumsspalte (vorher dytpe object) in ein Date-Time-Format. Überprüfung, ob nach der Konvertierung invalide Datumswerte existieren, was nicht der Fall ist.
Alle anderen Spalten haben den dtype float64 und werden so belassen.
Entfernung aller Datensätze, die nicht im Betrachtungszeitraum liegen (2020-2024). Durch Filterung über das Jahr werden 48 Einträge entfernt.

In [12]:
print(df_hourly.shape)
# Converting the column "Datum" from dtype object to DateTime format
# Invalid values are converted to NaT (Not a Time)
# Source: [3]
df_hourly['Datum'] = pd.to_datetime(df_hourly['Datum'], errors="coerce")
print(df_hourly.dtypes)



print(df_hourly['Datum'].isna().any())

# Create column "Year" from the Column "Datum" [4]

df_hourly['Jahr'] = df_hourly['Datum'].dt.year

# Filter only years between 2020 and 2024 

df_hourly = df_hourly[(df_hourly['Jahr'] >= 2020) & (df_hourly['Jahr'] <= 2024)]

# 48 rows deleted
print(df_hourly.shape) 

print(df_hourly["Jahr"].unique())

(43896, 4)
Datum                                datetime64[ns]
Leistung nicht erneuerbar (MW)              float64
Leistung erneuerbar (MW)                    float64
Day Ahead Auktion Preis (EUR/MWh)           float64
dtype: object
False
(43848, 5)
[2021 2020 2022 2023 2024]


c) Beurteilung der Datenqualität und notwenige Datenbereinigungsschritte. 
Qualität der Datumsdaten wurde durch Konvertierung schon überprüft. df_hourly enthält nur Werte für die Jahre 2020-2024.

In [16]:
# Check for missing values
print(df_hourly.isnull().sum())
print(df_hourly.notnull().sum())



Datum                                0
Leistung nicht erneuerbar (MW)       0
Leistung erneuerbar (MW)             0
Day Ahead Auktion Preis (EUR/MWh)    0
Jahr                                 0
dtype: int64
Datum                                43848
Leistung nicht erneuerbar (MW)       43848
Leistung erneuerbar (MW)             43848
Day Ahead Auktion Preis (EUR/MWh)    43848
Jahr                                 43848
dtype: int64


d) Erzeugung eines Dataframes df_daily

### Aufgabe 6 (Analyse von Stromtarif-Angeboten für Endkunden)
<a id = "aufgabe6"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a)  Einlesen der JSON-Preisvergleichdaten in einem DataFrame df_cust


In [ ]:
path = './Daten/Endkundenpreise/'

def extract_date_from_filepath(file):
    file = file.replace(path, "")
    # Concatenate the date out of fixed year 2024 and month/day from folder name
    return "2024-" + file[0:5]

def extract_data_from_json(file):
    assert file.endswith(".json"), "Keine json-Datei uebergeben"
    try:
        # Read and load the json file
        df_temp = pd.read_json(json_files[i])
        # Transpose the table: convert the rows to columns
        df_temp = df_temp.T
        # Index = Number of entry in the json file
        df_temp['Angebotsnummer'] = df_temp.index
        # Add date from filename as column
        df_temp['Datum'] = extract_date_from_filepath(file)
        
        return df_temp
    except:
        print("Datei konnte nicht gelesen werden.")


path = './Daten/Endkundenpreise/'
df_list = []
# https://www.tutorialspoint.com/python/os_listdir.htm
for folder in os.listdir(path):
    combined_path = os.path.join(path, folder, "*json")
    json_files = glob.glob(combined_path)

    for i in range(len(json_files)):
        df_temp = extract_data_from_json(json_files[i])
        df_list.append(df_temp)
        
df_cust = pd.concat(df_list)
df_cust


### Quellenverzeichnis
<a id = "quellenverzeichnis"></a>

[1] https://www.geeksforgeeks.org/how-to-read-all-csv-files-in-a-folder-in-pandas/

[2] https://statistikguru.de/python/python-auflisten-dateien-verzeichnis.html

[3] https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

[4] https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.year.html#pandas.Series.dt.year

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)